In [ ]:
#load data
"""
todo:
save model
"""
INPUT_TYPE=1 #0 = piece representation, 1=board representation with one-hot

import chess.pgn
import time
import numpy as np

pgn = open("pgn/ficsgamesdb_search_160938.pgn")

def list_moves(pgn,side):
    move_list=[]
    board_list=[]
    
    while True:
        game=chess.pgn.read_game(pgn)
        if game is None:
            break
        
        board = game.board()
        turn=1
        
        for move in game.mainline_moves():
            if turn%2==side:
                move_list.append(move)
                board_list.append(board.fen())
            
            board.push(move)
            turn=turn+1
            
    return move_list,board_list

def move_convert(move) :
    #converts move from png to move for the game
    move=str(move)
    dictionary=['a','b','c','d','e','f','g','h']
    return (7-(int(move[1])-1), dictionary.index(move[0]), 7-(int(move[3])-1), dictionary.index(move[2]))


def board_convert(board) :
    #converts fen board to board for the game
    i=0
    board_converted=[]
    
    while board[i]!=' ':
        if board[i]=='/':
            i=i+1
        if board[i] in ['1','2','3','4','5','6','7','8']:
            for j in range(int(board[i])):
                board_converted.append('0')
            i=i+1
        else:
            board_converted.append(board[i])
            i=i+1
    matrix=[]
    while board_converted != []:
        matrix.append(board_converted[:8])
        board_converted = board_converted[8:]
    return matrix

def board_flip(board):
    pieces1=['p','r','n','b','q','k']
    pieces2=['P','R','N','B','Q','K']
    for i in range(8):
        for j in range(8):
            if(board[i][j] in pieces1):
                board[i][j]=pieces2[pieces1.index(board[i][j])]
            else:
                if(board[i][j] in pieces2):
                    board[i][j]=pieces1[pieces2.index(board[i][j])]
    return board

def board_transform1(board_state):
# transforms board state into input for the neural network
    board_state_temp=[[0,0,0] for i in range(32)]
    
    pieces=['p','r','n','b','q','k','P','R','N','B','Q','K']
    amount=[8,2,2,2,1,1,8,2,2,2,1,1]
    amount_so_far=[0,0,0,0,0,0,0,0,0,0,0,0]
    
    #board_state=board_flip(board_state)
    
    for i in range(8):
        for j in range(8):
            for k in range(12):
                if(board_state[i][j]==pieces[k]):
                    board_state_temp[sum(amount[:k])+amount_so_far[k]]=[i/7,j/7,1]
                    amount_so_far[k]+=1
    
    for i in range(32):
        board_state_temp[i]=np.ravel(board_state_temp[i])
    
    return board_state_temp


def board_transform2(board_state):
# transforms board state into input for the neural network
    board_state_temp=[]
    for i in board_state:
        board_state_temp.append(list(i))
    pieces1=['p','r','n','b','q','k','P','R','N','B','Q','K']
    
    
    for i in range (8):
        for j in range (8):
            one_hot=[0]*12
            for k in range (12):
                if(board_state[i][j]==pieces1[k]):
                    one_hot[k]=1
                    board_state_temp[i][j]=one_hot
            if(board_state[i][j]=='0' or board_state[i][j]==0 ):
                board_state_temp[i][j]=one_hot
            

    return board_state_temp


def board_transform(board_state):
    if INPUT_TYPE==0:
        return board_transform1(board_state)
    if INPUT_TYPE==1:
        return board_transform2(board_state)

#loading and processing data



move_list, board_list=list_moves(pgn,1)

print(board_list[1])
#input
input_list=board_list
if(INPUT_TYPE==0):
    for i in range (len(board_list)):
        input_list[i]=np.ravel(board_transform1(board_convert(input_list[i])))

if(INPUT_TYPE==1):
    for i in range (len(board_list)):
        input_list[i]=np.ravel(board_transform2(board_convert(input_list[i])))

input_list=np.asarray(input_list)

#target
target_list=[]
from_target_list=[]
to_target_list=[]

for i in range (len(move_list)):
    move_list[i]=move_convert(move_list[i])
    #target_list.append(target_list[i][3]*8*8*8+target_list[i][2]*8*8+target_list[i][1]*8+target_list[i][0])
    from_target_list.append(move_list[i][0]*8+move_list[i][1])
    to_target_list.append(move_list[i][2]*8+move_list[i][3])

target_list=np.asarray(target_list)
from_target_list=np.asarray(from_target_list)
to_target_list=np.asarray(to_target_list)


#adding moved_from to the input of the moved_to model
to_input_list=[]
for i in range (len(input_list)):
    to_input_list.append(list(input_list[i]))
    to_input_list[i].append(from_target_list[i]/63)

to_input_list=np.asarray(to_input_list)



print(input_list[0])
print(to_input_list[1])

print(target_list.shape)

print(move_list[0])
print(from_target_list)
print(to_target_list)


In [ ]:
#make model

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

#building the moved from model
from_model = tf.keras.Sequential([
    tf.keras.layers.Dense(1000, activation='relu'),
    tf.keras.layers.Dense(8*8, activation='softmax')
])

from_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history1=from_model.fit(input_list, from_target_list, epochs=200, batch_size=20, validation_split=0.3)

probability_from_model = tf.keras.Sequential([from_model])







#building the moved to model
to_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(8*8, activation='softmax')
])

to_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history2=to_model.fit(to_input_list, to_target_list, epochs=200, batch_size=20, validation_split=0.3)

probability_from_model = tf.keras.Sequential([from_model])



def predict(board):
    #predicts the move taken in a board state
    board = (np.expand_dims(board,0))
    predictions_single = probability_from_model.predict(board)
    return predictions_single


In [ ]:
#plot

import matplotlib.pyplot as plt

#from model


history_dict = history1.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)


plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()



plt.plot(epochs, acc, 'b', label='Tanuló pontosság')
plt.plot(epochs, val_acc, 'r', label='Validációs pontosság')
#plt.title('Training and validation accuracy')
plt.ylim([0, 1])
plt.xlabel('Epoch')
plt.ylabel('Pontosság')
plt.legend(loc='lower right')

plt.show()


#to model

history_dict = history2.history

acc2 = history_dict['accuracy']
val_acc2 = history_dict['val_accuracy']
loss2 = history_dict['loss']
val_loss2 = history_dict['val_loss']

epochs = range(1, len(acc) + 1)


plt.plot(epochs, loss2, 'b', label='Training loss')
plt.plot(epochs, val_loss2, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()



plt.plot(epochs, acc2, 'b', label='Tanuló pontosság')
plt.plot(epochs, val_acc2, 'r', label='Validációs pontosság')
#plt.plot(epochs, acc, 'c', label='Tanuló pontosság')
#plt.plot(epochs, val_acc, 'm', label='Validációs pontosság')
#plt.title('Training and validation accuracy')
plt.ylim([0, 1])
plt.xlabel('Epoch')
plt.ylabel('Pontosság')
plt.legend(loc='best')

plt.show()

In [ ]:
#test


test=predict(input_list[0])

test=np.sort(test)
print(test)
print(type(test))
test=move_transform(test)
print(test)
print(test[0]*8+test[1])

In [ ]:
#save models
from_model.save('models1')
to_model.save('models2')

In [ ]:
#load models
from tensorflow import keras
from_saved_model = keras.models.load_model('models1')
to_saved_model = keras.models.load_model('models2')

def predict_saved(board):
    #predicts the move taken in a board state
    board = (np.expand_dims(board,0))
    predictions_single = saved_model.predict(board)
    return predictions_single

In [ ]:
"""
player functions to use in play_game method:
random_player - randomly chooses from available moves
human_player - takes keyboard input to choose from the list of available moves. input example : "1, 0, 2, 0" moves the
               chess piece from 1,0 coordinates to 2,0 coordinates
min_max_player - runs the min-max algorithm to choose the best available move
make_move - player trained with reinforced learning
"""
import random
import itertools
import numpy as np


def _new_board():
    """Return a emprty chess board we can use for simulating a game.
    Returns:
        8x8 tuple of ints
    """
    return (('r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'),
            ('p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'),
            (0, 0, 0, 0, 0, 0, 0, 0),
           (0, 0, 0, 0, 0, 0, 0, 0),
           (0, 0, 0, 0, 0, 0, 0, 0),
           (0, 0, 0, 0, 0, 0, 0, 0),
            ('P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'),
            ('R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R')
           )


def apply_move(board_state, move):
    """Returns a copy of the given board_state with the desired move applied.
    Args:
        board_state (3x3 tuple of int): The given board_state we want to apply the move to.
        move (int, int): The position we want to make the move in.
        side (int): The side we are making this move for, 1 for the first player, -1 for the second player.
    Returns:
        (3x3 tuple of int): A copy of the board_state with the given move applied for the given side.
    """
    from_x, from_y, to_x, to_y = move

    def get_tuples():
        temp = list(board_state)
        
        for x in range(8):
            for y in range(8):
                if(from_x == x and from_y == y):
                    chess_piece=temp[x][y]
                    temp[x]=list(temp[x])
                    temp[x][y]=0
                    temp[x]=tuple(temp[x])
                    
        for x in range(8):
            for y in range(8):
                if(to_x == x and to_y == y):
                    temp[x]=list(temp[x])
                    temp[x][y]=chess_piece
                    temp[x]=tuple(temp[x])
        
        
        return tuple(temp)
        
    return tuple(get_tuples())


def is_black(board_state, x, y):
    #returns if the piece on x,y coordinates is white or not
    if (board_state[x][y]=='p' or board_state[x][y]=='r' or board_state[x][y]=='n' or board_state[x][y]=='b' or board_state[x][y]=='q' or board_state[x][y]=='k') :
        return True
    else:
        return False
    
def is_white(board_state, x, y):
    #returns if the piece on x,y coordinates is black or not
    if (board_state[x][y]=='P' or board_state[x][y]=='R' or board_state[x][y]=='N' or board_state[x][y]=='B' or board_state[x][y]=='Q' or board_state[x][y]=='K') :
        return True
    else:
        return False
    
def available_moves(board_state, side):
    """Get all legal moves for the current board_state.
    Args:
        board_state: The board_state we want to check for valid moves.
    Returns:
        Generator of (int, int): All the valid moves that can be played in this position.
    """
    for x, y in itertools.product(range(8), range(8)):
        
        if (side == 1):
        #white player:
        
            if (board_state[x][y] == 'P'):
            #white pawn:
            
                if (x>0 and board_state[x-1][y] == 0): #forward 1
                    yield (x, y, x-1, y)
                if (x==6 and board_state[x-1][y] == 0 and board_state[x-2][y] == 0): #forward 2
                    yield (x, y, x-2, y)
                if (x>0 and y>0 and is_black(board_state, x-1, y-1)): # forward 1 left 1
                    yield (x, y, x-1, y-1)
                if (x>0 and y<7 and is_black(board_state, x-1, y+1)): # forward 1 right 1
                    yield (x, y, x-1, y+1)

                    
            if (board_state[x][y] == 'K'):
            #white king:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1],[1,0],[-1,0],[0,-1],[0,1]] #directions the chess piece can move in
                for i in directions:
                    if(x+i[0]<8 and x+i[0]>-1 and y+i[1]<8 and y+i[1]>-1):
                        if(is_white(board_state,x+i[0],y+i[1])==False):
                            yield(x, y, x+i[0], y+i[1])
                    
                    
            if (board_state[x][y] == 'R'):
            #white rook:
            
                directions=[[1,0],[-1,0],[0,-1],[0,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_white(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_black(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
 
            if (board_state[x][y] == 'B'):
            #white bishop:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_white(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_black(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
                                
                                
                                
            if (board_state[x][y] == 'Q'):
            #white queen:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1],[1,0],[-1,0],[0,-1],[0,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_white(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_black(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
                                
            if (board_state[x][y] == 'N'):
            #white knight:
            
                directions=[[1,2],[1,-2],[-1,2],[-1,-2],[2,1],[2,-1],[-2,1],[-2,-1]]
                for i in directions:
                    if(x+i[0]<8 and x+i[0]>-1 and y+i[1]<8 and y+i[1]>-1):
                        if(is_white(board_state,x+i[0],y+i[1])==False):
                            yield(x, y, x+i[0], y+i[1])


        if (side == -1):
        #black player:
            
            if (board_state[x][y] == 'p' and side == -1):
                #black pawn:
            
                if (x<7 and board_state[x+1][y] == 0): #forward 1
                    yield (x, y, x+1, y)
                if (x==1 and board_state[x+1][y] == 0 and is_black(board_state, x+2, y)==False): #forward 2
                    yield (x, y, x+2, y)
                if (x<7 and y>0 and is_white(board_state, x+1, y-1)): # forward 1 left 1
                    yield (x, y, x+1, y-1)
                if (x<7 and y<7 and is_white(board_state, x+1, y+1)): # forward 1 right 1
                    yield (x, y, x+1, y+1)

                    
                    
            if (board_state[x][y] == 'k'):
            #white king:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1],[1,0],[-1,0],[0,-1],[0,1]]
                for i in directions:
                    if(x+i[0]<8 and x+i[0]>-1 and y+i[1]<8 and y+i[1]>-1):
                        if(is_black(board_state,x+i[0],y+i[1])==False):
                            yield(x, y, x+i[0], y+i[1])
                            
                            
                    

            if (board_state[x][y] == 'r'):
            #black rook:
            
                directions=[[1,0],[-1,0],[0,-1],[0,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_black(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_white(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
                                
            if (board_state[x][y] == 'b'):
            #black bishop:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_black(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_white(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
                                
                                
            if (board_state[x][y] == 'q'):
            #black queen:
            
                directions=[[1,1],[-1,-1],[1,-1],[-1,1],[1,0],[-1,0],[0,-1],[0,1]]
                for i in directions:
                        blocked=0
                        coordinates=[x,y]
                        while(blocked==0 and coordinates[0]+i[0]<8 and coordinates[0]+i[0]>-1 and coordinates[1]+i[1]<8 and coordinates[1]+i[1]>-1):
                            coordinates[0]+=i[0]
                            coordinates[1]+=i[1]
                            if(is_black(board_state,coordinates[0],coordinates[1])):
                                blocked=1
                            else:
                                if(is_white(board_state,coordinates[0],coordinates[1])):
                                    blocked=1
                                yield (x, y, coordinates[0], coordinates[1])
                                
                                
            if (board_state[x][y] == 'n'):
            #black knight:
            
                directions=[[1,2],[1,-2],[-1,2],[-1,-2],[2,1],[2,-1],[-2,1],[-2,-1]]
                for i in directions:
                    if(x+i[0]<8 and x+i[0]>-1 and y+i[1]<8 and y+i[1]>-1):
                        if(is_black(board_state,x+i[0],y+i[1])==False):
                            yield(x, y, x+i[0], y+i[1])

                            

def has_winner(board_state):
    """Determine if a player has won on the given board_state.
    Args:
        board_state (8x8 tuple of int): The current board_state we want to evaluate.
    Returns:
        int: 1 if player one has won, -1 if player 2 has won, otherwise 0.
    """
    black_king=0
    white_king=0
    
    for x in range(8):
        for y in range(8):
            if(board_state[x][y] == 'K'):
                white_king=1
            if(board_state[x][y] == 'k'):
                black_king=1       
    
    if black_king==0:
        return 1
    if white_king==0:
        return -1
    
    return 0  # no one has won, return 0 for a draw

def print_board(board_state,move):
  for i in range (len(board_state)):
    for j in range (len(board_state[0])):
      if((move[0]==i and move[1]==j) or (move[2]==i and move[3]==j)):
        print("\033[1;33m%s" %board_state[i][j], end = '  ')
      else:
        print("\033[1;30m%s" %board_state[i][j], end = '  ')
    print("\n")

#converting board state to fen
from more_itertools import run_length


def convert_cell(value):
    if value == 0:
        return None
    else:
        return value


def convert_rank(rank):
    return ''.join(
        value * count if value else str(count)
        for value, count in run_length.encode(map(convert_cell, rank))
    )


def fen_from_board(board):
    return '/'.join(map(convert_rank, board)) + ' w KQkq - 0 1'
 
    
def play_game(plus_player_func, minus_player_func, log=0, max_game_length=100):
    """Run a single game of tic-tac-toe until the end, using the provided function args to determine the moves for each
    player.
    Args:
        plus_player_func ((board_state(3 by 3 tuple of int), side(int)) -> move((int, int))): Function that takes the
            current board_state and side this player is playing, and returns the move the player wants to play.
        minus_player_func ((board_state(3 by 3 tuple of int), side(int)) -> move((int, int))): Function that takes the
            current board_state and side this player is playing, and returns the move the player wants to play.
        log (bool): If True progress is logged to console, defaults to False
    Returns:
        int: 1 if the plus_player_func won, -1 if the minus_player_func won and 0 for a draw
    """
    board_state = _new_board()
    player_turn = 1
    turn = 0
    points=[]
    while True:
        _available_moves = list(available_moves(board_state, player_turn))
        
        turn=turn+1
        
        if len(_available_moves) == 0:
            # draw
            if log:
                print("no moves left, game ended a draw")
            
            return 0, points
        if player_turn > 0:
            move = plus_player_func(board_state, 1)
        else:
            move = minus_player_func(board_state, -1)

        if move not in _available_moves:
            # if a player makes an invalid move the other player wins
            if log:
                print("illegal move ", move)
            
            if player_turn > 0:
                points.append(player_turn*(-100))
            return -player_turn, points
        
        pre_move_point = position_points(board_state)
        
        board_state = apply_move(board_state, move)
        
        post_move_point = position_points(board_state)
        
        if player_turn > 0:
            points.append(post_move_point-pre_move_point)
        
        if log==1:
            
            print(fen_from_board(board_state))
            print_board(board_state,move)
            print("___________")
            

        if log==2:
            
            print(np.matrix(board_state))
            print("___________")

        winner = has_winner(board_state)
        if winner != 0:
            if log:
                print("we have a winner, side: %s" % player_turn)
            return winner, points
        player_turn = -player_turn
        if turn == max_game_length:
            return 0, points,


def move_transform(move):
# transforms output from the network into a move

    m = np.argmax(list(move))

    x1 = m % 8
    m = (m-x1)/8
    y1 = m % 8
    m = (m-y1)/8

    return x1,y1
        

def random_player(board_state, side):
    """A player func that can be used in the play_game method. Given a board state it chooses a move randomly from the
    valid moves in the current state.
    Args:
        board_state (3x3 tuple of int): The current state of the board
        _: the side this player is playing, not used in this function because we are simply choosing the moves randomly
    Returns:
        (int, int): the move we want to play on the current board
    """
    moves = list(available_moves(board_state, side))
    return random.choice(moves)

def pro_predict_player(board_state, side):
    board_state_temp=board_transform(board_state)
    board_state_temp=np.ravel(board_state_temp)
    test=predict(board_state_temp)
    test=move_transform(test)
    return test

def human_player(board_state,side):
    moves = list(available_moves(board_state,side))
    print(moves)
    move = 0
    while(move not in list(available_moves(board_state,side))):
        move=input()
        move = tuple(map(int, move.split(', '))) 
    return(move)


def position_points(board_state):
    pieces=['p','r','n','b','q','k','P','R','N','B','Q','K']
    points=[-1,-5,-3,-3,-9,-100,1,5,3,3,9,100]
    point=0
    for i in range(8):
        for j in range(8):
            for k in range(12):
                if(board_state[i][j]==pieces[k]):
                    point=point+points[k]
    return point
                    

if __name__ == '__main__':
    # example of playing a game
    play_game(random_player,random_player, log=1)